## 이미지 다루기

이미지는 픽셀 단위의 높이와 너비를 가지는 표준적인 그리드에 나열된 복수 개의 스칼라 값 모음으로 표현된다.

In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50)

In [2]:
import imageio

img_arr= imageio.imread('/content/drive/MyDrive/deep_learning_with_pytorch/data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

# 너비와 높이에 해당하는 두 개의 공간 정보와 빨,초,파 채널에 해당하는 세 번째 차원까지 합하면 3차원 배열 같은 넘파이 객체다.

<ipython-input-2-1b1cf222ed2c>:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr= imageio.imread('/content/drive/MyDrive/deep_learning_with_pytorch/data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

## 레이아웃 변경하기

In [3]:
#구식의 차원 레이아웃에서 적절한 레이아웃을 얻기 위해 새로운 차원으로 변경하려면 텐서의 permute 메소드를 사용한다.

img = torch.from_numpy(img_arr)
out = img.permute(2,0,1)

우리가 다룰 신경망의 입력으로 사용할 여러 장의 이미지 데이터셋을 만들어보자. 


첫 번째 차원에 여러 이미지를 batch로 넣어 N * C * H * W 텐서로 저장하자.

In [5]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

#배치의 높이, 너비 256픽셀, RGB 이미지 세 개로 이뤄진 배치

In [6]:
import os

data_dir= '/content/drive/MyDrive/deep_learning_with_pytorch/data/p1ch4/image-cats'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']

for i, filename in enumerate(filenames):
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  img_t = img_t[:3]
  batch[i] = img_t

<ipython-input-6-466c097e7369>:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


## 데이터 정규화

In [8]:
batch = batch.float()
batch /= 255.0

# 픽셀 값을 부호 없는 8비트 정수의 최댓값인 255로 나눈다.

In [9]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

#전체 데이터셋을 다루는 방법은 아직 모르므로, 하나의 배치 이미지 묶음만 정규화했다.
#실제로 이미지로 작업할 때는, 미리 모든 훈련 데이터에 대한 평균과 표준 편차를 계산해서 이렇게 사전에 계산되고 고정된 값으로 빼고 나눈 방법이 좋은 관례다.